In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def delete_outlier(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]    
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_outlier = df[(df['SMP'] < lower_bound) | (df['SMP'] > upper_bound)]
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]
    
    # if(len(df_outlier)>0):
    #     print('Ada Outlier')

    # df_grouped = df_cleaned[['HARI', 'JAM', 'SMP']]
    # df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_outlier, df_cleaned


In [3]:
path_1 = '../Data/Preprocessing/Data Balonggandu/2023/2_input_balonggandu_2023.xlsx'

df = read_excel(path_1)
df.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,1,Sunday,00:00,103,25,14,142,82.4,25,35.0,142
1,2023-01-01 00:15:00,2023,1,1,1,Sunday,00:15,111,16,12,139,88.8,16,30.0,135
2,2023-01-01 00:30:00,2023,1,1,1,Sunday,00:30,114,26,6,146,91.2,26,15.0,132
3,2023-01-01 00:45:00,2023,1,1,1,Sunday,00:45,76,24,11,111,60.8,24,27.5,112
4,2023-01-01 01:00:00,2023,1,1,1,Sunday,01:00,73,24,7,104,58.4,24,17.5,100


In [4]:
df_waktu = df[['HARI','JAM']]
df_waktu = df_waktu.groupby(['HARI', 'JAM']).mean().reset_index()
df_waktu.head()

,HARI,JAM
0,Friday,00:00
1,Friday,00:15
2,Friday,00:30
3,Friday,00:45
4,Friday,01:00


In [5]:
all_df_outlier = []
all_df_cleaned = []

for i, row in df_waktu.iterrows():
    df_outlier,  df_cleaned = delete_outlier(df, row['HARI'], row['JAM'])
    all_df_outlier.append(df_outlier)
    all_df_cleaned.append(df_cleaned)

# gabungkan data
df_outlier_combined = pd.concat(all_df_outlier)
df_cleaned_combined = pd.concat(all_df_cleaned)

In [6]:
df_outlier_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
5184,2023-02-24,2023,2,4,24,Friday,00:00,5,38,33,76,4.0,38,82.5,124
8544,2023-03-31,2023,3,5,31,Friday,00:00,60,48,75,183,48.0,48,187.5,284
11904,2023-05-05,2023,5,1,5,Friday,00:00,92,52,58,202,73.6,52,145.0,271
12576,2023-05-12,2023,5,2,12,Friday,00:00,71,42,68,181,56.8,42,170.0,269
13248,2023-05-19,2023,5,3,19,Friday,00:00,68,52,67,187,54.4,52,167.5,274


In [7]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
480,2023-01-06,2023,1,1,6,Friday,00:00,73,35,40,148,58.4,35,100.0,193
1152,2023-01-13,2023,1,2,13,Friday,00:00,69,47,34,150,55.2,47,85.0,187
1824,2023-01-20,2023,1,3,20,Friday,00:00,50,38,40,128,40.0,38,100.0,178
2496,2023-01-27,2023,1,4,27,Friday,00:00,46,40,49,135,36.8,40,122.5,199
3168,2023-02-03,2023,2,1,3,Friday,00:00,40,39,38,117,32.0,39,95.0,166


In [8]:
df_outlier_combined = df_outlier_combined.sort_values(by='WAKTU').reset_index(drop=True)
df_cleaned_combined = df_cleaned_combined.sort_values(by='WAKTU').reset_index(drop=True)

In [9]:
df_cleaned_combined.to_excel('../Data/Preprocessing/Data Balonggandu/2023/3_clean_balonggandu_2023.xlsx', index=False)
df_outlier_combined.to_excel('../Data/Preprocessing/Data Balonggandu/2023/3_outliers_balonggandu_2023.xlsx', index=False)

In [10]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,1,Sunday,00:00,103,25,14,142,82.4,25,35.0,142
1,2023-01-01 00:15:00,2023,1,1,1,Sunday,00:15,111,16,12,139,88.8,16,30.0,135
2,2023-01-01 00:30:00,2023,1,1,1,Sunday,00:30,114,26,6,146,91.2,26,15.0,132
3,2023-01-01 00:45:00,2023,1,1,1,Sunday,00:45,76,24,11,111,60.8,24,27.5,112
4,2023-01-01 01:15:00,2023,1,1,1,Sunday,01:15,86,28,9,123,68.8,28,22.5,119


In [11]:
df_smp = df_cleaned_combined[['WAKTU','TAHUN', 'BULAN', 'PEKAN' ,'TANGGAL', 'HARI','JAM','MOTOR','MOBIL','TRUK/BUS',
                              'SMP_MTR','SMP_MBL','SMP_TRK','SMP']]

In [12]:
df_smp.head(8)

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,1,Sunday,00:00,103,25,14,82.4,25,35.0,142
1,2023-01-01 00:15:00,2023,1,1,1,Sunday,00:15,111,16,12,88.8,16,30.0,135
2,2023-01-01 00:30:00,2023,1,1,1,Sunday,00:30,114,26,6,91.2,26,15.0,132
3,2023-01-01 00:45:00,2023,1,1,1,Sunday,00:45,76,24,11,60.8,24,27.5,112
4,2023-01-01 01:15:00,2023,1,1,1,Sunday,01:15,86,28,9,68.8,28,22.5,119
5,2023-01-01 01:30:00,2023,1,1,1,Sunday,01:30,74,20,10,59.2,20,25.0,104
6,2023-01-01 02:00:00,2023,1,1,1,Sunday,02:00,46,21,6,36.8,21,15.0,73
7,2023-01-01 02:30:00,2023,1,1,1,Sunday,02:30,51,21,10,40.8,21,25.0,87


In [13]:
# Round down the WAKTU column to the nearest hour
df_smp_jam = df_smp.copy()
df_smp_jam['JAM'] = df_smp['WAKTU'].dt.floor('H')

# Drop the original WAKTU column if it's not needed anymore
df_smp_jam = df_smp_jam.drop(columns=['WAKTU'])

# Format the 'JAM' column to show only hour and minute
df_smp_jam['JAM'] = df_smp_jam['JAM'].dt.strftime('%H:%M')

In [14]:
df_smp_jam.head(8)

,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023,1,1,1,Sunday,00:00,103,25,14,82.4,25,35.0,142
1,2023,1,1,1,Sunday,00:00,111,16,12,88.8,16,30.0,135
2,2023,1,1,1,Sunday,00:00,114,26,6,91.2,26,15.0,132
3,2023,1,1,1,Sunday,00:00,76,24,11,60.8,24,27.5,112
4,2023,1,1,1,Sunday,01:00,86,28,9,68.8,28,22.5,119
5,2023,1,1,1,Sunday,01:00,74,20,10,59.2,20,25.0,104
6,2023,1,1,1,Sunday,02:00,46,21,6,36.8,21,15.0,73
7,2023,1,1,1,Sunday,02:00,51,21,10,40.8,21,25.0,87


In [15]:
# Group by the rounded-down hour and sum the SMP column
df_smp_jam = df_smp_jam.groupby(['TAHUN', 'BULAN', 'PEKAN', 'TANGGAL', 'HARI', 'JAM'], as_index=False).mean().round()

In [16]:
df_smp_jam.head(8)

,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023,1,1,1,Sunday,00:00,101.0,23.0,11.0,81.0,23.0,27.0,130.0
1,2023,1,1,1,Sunday,01:00,80.0,24.0,10.0,64.0,24.0,24.0,112.0
2,2023,1,1,1,Sunday,02:00,48.0,21.0,8.0,39.0,21.0,20.0,80.0
3,2023,1,1,1,Sunday,03:00,43.0,18.0,10.0,34.0,18.0,24.0,77.0
4,2023,1,1,1,Sunday,04:00,53.0,16.0,9.0,42.0,16.0,22.0,81.0
5,2023,1,1,1,Sunday,05:00,104.0,24.0,10.0,84.0,24.0,26.0,134.0
6,2023,1,1,1,Sunday,07:00,229.0,72.0,24.0,183.0,72.0,61.0,317.0
7,2023,1,1,1,Sunday,08:00,200.0,100.0,30.0,160.0,100.0,76.0,335.0


In [17]:
# List kolom yang akan dikalikan 4
columns_to_multiply = ['MOTOR', 'MOBIL', 'TRUK/BUS', 'SMP_MTR', 'SMP_MBL', 'SMP_TRK', 'SMP']

# Mengalikan nilai kolom dengan 4
df_smp_jam[columns_to_multiply] = df_smp_jam[columns_to_multiply] * 4

In [18]:
df_smp_jam.head(8)

,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023,1,1,1,Sunday,00:00,404.0,92.0,44.0,324.0,92.0,108.0,520.0
1,2023,1,1,1,Sunday,01:00,320.0,96.0,40.0,256.0,96.0,96.0,448.0
2,2023,1,1,1,Sunday,02:00,192.0,84.0,32.0,156.0,84.0,80.0,320.0
3,2023,1,1,1,Sunday,03:00,172.0,72.0,40.0,136.0,72.0,96.0,308.0
4,2023,1,1,1,Sunday,04:00,212.0,64.0,36.0,168.0,64.0,88.0,324.0
5,2023,1,1,1,Sunday,05:00,416.0,96.0,40.0,336.0,96.0,104.0,536.0
6,2023,1,1,1,Sunday,07:00,916.0,288.0,96.0,732.0,288.0,244.0,1268.0
7,2023,1,1,1,Sunday,08:00,800.0,400.0,120.0,640.0,400.0,304.0,1340.0


In [19]:
df_lhrt = df_smp_jam.copy()

In [20]:
df_lhrt = df_lhrt.drop(columns=['TAHUN','BULAN','TANGGAL','PEKAN'])

In [21]:
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Sunday,00:00,404.0,92.0,44.0,324.0,92.0,108.0,520.0
1,Sunday,01:00,320.0,96.0,40.0,256.0,96.0,96.0,448.0
2,Sunday,02:00,192.0,84.0,32.0,156.0,84.0,80.0,320.0
3,Sunday,03:00,172.0,72.0,40.0,136.0,72.0,96.0,308.0
4,Sunday,04:00,212.0,64.0,36.0,168.0,64.0,88.0,324.0


In [22]:
# df_lhrt  = df_lhrt.groupby(['PEKAN', 'HARI', 'JAM'], as_index=False).mean().round()
df_lhrt  = df_lhrt.groupby(['HARI', 'JAM'], as_index=False).mean().round()

df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Friday,00:00,182.0,158.0,161.0,146.0,158.0,401.0,705.0
1,Friday,01:00,135.0,170.0,157.0,108.0,170.0,392.0,670.0
2,Friday,02:00,112.0,167.0,155.0,89.0,167.0,387.0,644.0
3,Friday,03:00,115.0,163.0,144.0,92.0,163.0,359.0,615.0
4,Friday,04:00,242.0,159.0,138.0,194.0,159.0,345.0,697.0


In [23]:
df_lhrt.to_excel('../Data/Preprocessing/Data Balonggandu/2023/3_lhrt_balonggandu_2023.xlsx', index=False)

In [24]:
len(df_lhrt)

168